In [1]:
import sys
sys.path.insert(1, './gym_powertac')

from powertac_wm import PowerTAC_WM
import gym
import datetime
import json
import numpy as np
import pandas as pd

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

from ReplayBuffer import ReplayBuffer
from ActorNetwork import ActorNetwork
from CriticNetwork import CriticNetwork
from OU import OU

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
model_storage_path = "ddpg_v0.0" 

In [3]:
BUFFER_SIZE = 100000
BATCH_SIZE = 64
EPOCHS = 10000
GAMMA = 0.99
TAU = 0.001     #Target Network HyperParameters
LRA = 0.0001    #Learning rate for Actor
LRC = 0.001     #Lerning rate for Critic

'''
Action : [
           limitprice belongs to R
         ] (one output nuerons)
'''
action_dim = 1

'''
State : [
          Proximity (24)
          Required_Quantity (1)
        ] (total 25 input nuerons)
'''
state_dim = 25

np.random.seed(1337)
EXPLORE = 100000.0

step = 0
epsilon = 1

ou = OU()       #Ornstein-Uhlenbeck Process

In [4]:
config = tf.ConfigProto(
    device_count={'GPU': 1},
    intra_op_parallelism_threads=1,
    allow_soft_placement=True
)

config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.6

session = tf.Session(config=config)

In [5]:
actor = ActorNetwork(session, state_dim, action_dim, BATCH_SIZE, TAU, LRA)
critic = CriticNetwork(session, state_dim, action_dim, BATCH_SIZE, TAU, LRC)
replay_buffer = pd.read_csv('replay_buffer_norm_big.csv', header=None)


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 25)]              0         
_________________________________________________________________
dense (Dense)                (None, 400)               10400     
_________________________________________________________________
dense_1 (Dense)              (None, 300)               120300    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 301       
_________________________________________________________________
concatenate (Concatenate)    (None, 1)                 0         
Total params: 131,001
Trainable params: 131,001
Non-trainable params: 0
_________________________________________________________________
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output S

In [6]:
def train_ddpg_network():

    for epoch in range(EPOCHS):

        print("Epoch ", (epoch+1))
        print("-"*12)
        loss = 0

        #Do the batch update
        batch = replay_buffer.sample(n=BATCH_SIZE)
        states = np.asarray(batch[batch.columns[0:25]])
        actions = np.asarray(batch[batch.columns[25:26]])
        rewards = np.asarray(batch[batch.columns[26:27]])
        new_states = np.asarray(batch[batch.columns[27:52]])
        terminals = np.asarray(batch[batch.columns[52:53]])

        y_t = np.zeros([BATCH_SIZE,1])

#         print("States", states.shape)
#         print("Actions", actions.shape)
#         print("Rewards", rewards.shape)
#         print("New_States", new_states.shape)

        with session.as_default():
            with session.graph.as_default():

                target_q_values = critic.target_model.predict([new_states, actor.target_model.predict(new_states)])
                # print("Target_Q_Values", target_q_values.shape)

                for k in range(BATCH_SIZE):
                    if terminals[k] == 1:
                        y_t[k] = rewards[k]
                    else:
                        y_t[k] = rewards[k] + GAMMA*target_q_values[k]

                # print("Bellman Rewards", y_t)
                loss += critic.model.train_on_batch([states,actions], y_t)
                print("Loss", loss)
                a_for_grad = actor.model.predict(states)      # This may not be required, a_for_grad should be replaced by actions ##### Check PENDING #####
                # print("a_for_grad", a_for_grad)
                grads = critic.gradients(states, a_for_grad)       # a_for_grad is replaced by actions ##### Check PENDING #####   shape ERROR 
                # print("grads", grads)
                actor.train(states, grads)
                actor.target_train()
                critic.target_train()

    print("Training Completed !!!")

In [7]:
train_ddpg_network()

Epoch  1
------------


/home/sanjay/anaconda3/envs/powertac_ddpg/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Loss 0.8531884551048279
Epoch  2
------------
Loss 0.6908241510391235
Epoch  3
------------
Loss 0.39998823404312134
Epoch  4
------------
Loss 0.4350718855857849
Epoch  5
------------
Loss 3.5212676525115967
Epoch  6
------------
Loss 0.24486935138702393
Epoch  7
------------
Loss 0.2096559703350067
Epoch  8
------------
Loss 1.9403316974639893
Epoch  9
------------
Loss 0.19459763169288635
Epoch  10
------------
Loss 0.1984574794769287
Epoch  11
------------
Loss 1.4283291101455688
Epoch  12
------------
Loss 0.8883281946182251
Epoch  13
------------
Loss 0.10752344876527786
Epoch  14
------------
Loss 0.17259609699249268
Epoch  15
------------
Loss 0.04643946513533592
Epoch  16
------------
Loss 0.782195508480072
Epoch  17
------------
Loss 0.13816556334495544
Epoch  18
------------
Loss 0.10909489542245865
Epoch  19
------------
Loss 0.14576959609985352
Epoch  20
------------
Loss 0.17201407253742218
Epoch  21
------------
Loss 2.332718849182129
Epoch  22
------------
Loss 0.232876

Epoch  181
------------
Loss 0.0446857213973999
Epoch  182
------------
Loss 0.22801664471626282
Epoch  183
------------
Loss 0.26509302854537964
Epoch  184
------------
Loss 0.24420467019081116
Epoch  185
------------
Loss 3.3928511142730713
Epoch  186
------------
Loss 0.1519576758146286
Epoch  187
------------
Loss 0.10507694631814957
Epoch  188
------------
Loss 0.06580226123332977
Epoch  189
------------
Loss 0.025959040969610214
Epoch  190
------------
Loss 0.9831013679504395
Epoch  191
------------
Loss 0.22406847774982452
Epoch  192
------------
Loss 0.30414479970932007
Epoch  193
------------
Loss 0.20135417580604553
Epoch  194
------------
Loss 0.4914175271987915
Epoch  195
------------
Loss 0.033429261296987534
Epoch  196
------------
Loss 0.06823679059743881
Epoch  197
------------
Loss 0.2620849013328552
Epoch  198
------------
Loss 0.16404542326927185
Epoch  199
------------
Loss 0.5277900695800781
Epoch  200
------------
Loss 0.695578932762146
Epoch  201
------------
Los

Epoch  352
------------
Loss 0.0749017596244812
Epoch  353
------------
Loss 0.07672533392906189
Epoch  354
------------
Loss 0.07654831558465958
Epoch  355
------------
Loss 0.41920968890190125
Epoch  356
------------
Loss 0.030812358483672142
Epoch  357
------------
Loss 0.07769754528999329
Epoch  358
------------
Loss 0.0834665447473526
Epoch  359
------------
Loss 0.9467074871063232
Epoch  360
------------
Loss 0.08738689869642258
Epoch  361
------------
Loss 0.5971137285232544
Epoch  362
------------
Loss 0.1515057235956192
Epoch  363
------------
Loss 0.15645289421081543
Epoch  364
------------
Loss 0.15842637419700623
Epoch  365
------------
Loss 0.6536693572998047
Epoch  366
------------
Loss 0.12784840166568756
Epoch  367
------------
Loss 0.07665283232927322
Epoch  368
------------
Loss 0.5907559990882874
Epoch  369
------------
Loss 0.03642549738287926
Epoch  370
------------
Loss 0.15209639072418213
Epoch  371
------------
Loss 0.20041874051094055
Epoch  372
------------
Lo

...

Epoch  9180
------------
Loss 0.1262878179550171
Epoch  9181
------------
Loss 0.2953653335571289
Epoch  9182
------------
Loss 1.109511137008667
Epoch  9183
------------
Loss 0.1736692488193512
Epoch  9184
------------
Loss 0.2735983729362488
Epoch  9185
------------
Loss 0.20017512142658234
Epoch  9186
------------
Loss 0.3109394311904907
Epoch  9187
------------
Loss 0.14290201663970947
Epoch  9188
------------
Loss 0.15304778516292572
Epoch  9189
------------
Loss 0.1845923215150833
Epoch  9190
------------
Loss 0.12132999300956726
Epoch  9191
------------
Loss 0.8912690877914429
Epoch  9192
------------
Loss 0.12952852249145508
Epoch  9193
------------
Loss 0.0964934378862381
Epoch  9194
------------
Loss 0.15512733161449432
Epoch  9195
------------
Loss 0.21583159267902374
Epoch  9196
------------
Loss 0.15532734990119934
Epoch  9197
------------
Loss 0.12849292159080505
Epoch  9198
------------
Loss 0.24018333852291107
Epoch  9199
------------
Loss 0.8455203175544739
Epoch  9200

Epoch  9350
------------
Loss 0.30649518966674805
Epoch  9351
------------
Loss 0.33029109239578247
Epoch  9352
------------
Loss 0.3086060881614685
Epoch  9353
------------
Loss 0.5172970294952393
Epoch  9354
------------
Loss 0.6164948344230652
Epoch  9355
------------
Loss 0.11750900745391846
Epoch  9356
------------
Loss 0.1902523636817932
Epoch  9357
------------
Loss 0.8364505171775818
Epoch  9358
------------
Loss 0.4397704601287842
Epoch  9359
------------
Loss 0.07444624602794647
Epoch  9360
------------
Loss 2.540926933288574
Epoch  9361
------------
Loss 0.34094908833503723
Epoch  9362
------------
Loss 0.13097506761550903
Epoch  9363
------------
Loss 1.0126309394836426
Epoch  9364
------------
Loss 0.6234917044639587
Epoch  9365
------------
Loss 0.43119412660598755
Epoch  9366
------------
Loss 0.1306653916835785
Epoch  9367
------------
Loss 0.15321843326091766
Epoch  9368
------------
Loss 0.20827239751815796
Epoch  9369
------------
Loss 0.22166317701339722
Epoch  9370

------------
Loss 0.38719919323921204
Epoch  9517
------------
Loss 0.11568906903266907
Epoch  9518
------------
Loss 0.26448118686676025
Epoch  9519
------------
Loss 0.23624449968338013
Epoch  9520
------------
Loss 0.24484659731388092
Epoch  9521
------------
Loss 0.1550886631011963
Epoch  9522
------------
Loss 0.23382706940174103
Epoch  9523
------------
Loss 0.6649330854415894
Epoch  9524
------------
Loss 0.200228750705719
Epoch  9525
------------
Loss 0.22757896780967712
Epoch  9526
------------
Loss 0.09578344225883484
Epoch  9527
------------
Loss 0.11235779523849487
Epoch  9528
------------
Loss 0.2012350857257843
Epoch  9529
------------
Loss 0.15084286034107208
Epoch  9530
------------
Loss 0.18738165497779846
Epoch  9531
------------
Loss 0.3212513029575348
Epoch  9532
------------
Loss 0.12097911536693573
Epoch  9533
------------
Loss 0.178231343626976
Epoch  9534
------------
Loss 0.7113547325134277
Epoch  9535
------------
Loss 0.325906366109848
Epoch  9536
-----------

Loss 0.20239943265914917
Epoch  9687
------------
Loss 0.1967313289642334
Epoch  9688
------------
Loss 0.39500343799591064
Epoch  9689
------------
Loss 0.07635653764009476
Epoch  9690
------------
Loss 0.8158976435661316
Epoch  9691
------------
Loss 0.8517305850982666
Epoch  9692
------------
Loss 0.5069370865821838
Epoch  9693
------------
Loss 0.21618428826332092
Epoch  9694
------------
Loss 0.12318427860736847
Epoch  9695
------------
Loss 0.11902246624231339
Epoch  9696
------------
Loss 0.48539531230926514
Epoch  9697
------------
Loss 0.12225844711065292
Epoch  9698
------------
Loss 0.10778608918190002
Epoch  9699
------------
Loss 0.1556556075811386
Epoch  9700
------------
Loss 0.15394897758960724
Epoch  9701
------------
Loss 0.10814809054136276
Epoch  9702
------------
Loss 0.28332284092903137
Epoch  9703
------------
Loss 0.08864058554172516
Epoch  9704
------------
Loss 0.4581349492073059
Epoch  9705
------------
Loss 0.7538765072822571
Epoch  9706
------------
Loss 0.

Loss 1.0743002891540527
Epoch  9857
------------
Loss 0.10169264674186707
Epoch  9858
------------
Loss 1.3182145357131958
Epoch  9859
------------
Loss 0.23626717925071716
Epoch  9860
------------
Loss 0.23545154929161072
Epoch  9861
------------
Loss 0.018896669149398804
Epoch  9862
------------
Loss 0.1209922507405281
Epoch  9863
------------
Loss 0.48831719160079956
Epoch  9864
------------
Loss 0.5556071400642395
Epoch  9865
------------
Loss 0.17722171545028687
Epoch  9866
------------
Loss 0.09622372686862946
Epoch  9867
------------
Loss 0.32249560952186584
Epoch  9868
------------
Loss 0.17219865322113037
Epoch  9869
------------
Loss 0.28887277841567993
Epoch  9870
------------
Loss 0.27184319496154785
Epoch  9871
------------
Loss 0.23536089062690735
Epoch  9872
------------
Loss 0.7985579967498779
Epoch  9873
------------
Loss 0.3373520076274872
Epoch  9874
------------
Loss 2.6206893920898438
Epoch  9875
------------
Loss 0.5303245782852173
Epoch  9876
------------
Loss 0.

In [8]:
def choose_Action(states):

        actions = list()

        for state in states:

            try:

                # self.epsilon -= 1.0 / self.EXPLORE
                # a_t = np.zeros([self.action_dim])
                # noise_t = np.zeros([self.action_dim])

                with session.as_default():
                    with session.graph.as_default():

                        a_t_original = actor.model.predict(state.reshape(1, state.shape[0]))[0].tolist()
                        # noise_t[0] = max(self.epsilon, 0) * self.ou.function(a_t_original[0],  0.0 , 0.60, 0.30)  # decide theta, sigma and mu for limitprice

                        # a_t[0] = a_t_original[0] + noise_t[0]
                        # a_t[1] = a_t_original[1] + noise_t[1]

                        # print(a_t_original)
                        actions.append(list(a_t_original))

            except Exception as e:
                print(e)

        return actions

In [9]:
batch = replay_buffer.sample(n=BATCH_SIZE)
states = np.asarray(batch[batch.columns[0:25]])

lps = choose_Action(states)

In [11]:
for lp in lps:
    print(lp)

[-0.4071735143661499]
[-0.9392122030258179]
[-1.1224138736724854]
[-0.4415271282196045]
[-0.7673320770263672]
[-0.6945095062255859]
[-1.4837603569030762]
[-0.8714370727539062]
[-0.6346230506896973]
[-0.7349237203598022]
[-0.9405196905136108]
[-0.5907711982727051]
[-0.7301993370056152]
[-0.7995903491973877]
[-0.9714541435241699]
[-0.9748969078063965]
[-1.0544447898864746]
[-0.5786714553833008]
[-0.5149672031402588]
[-1.140999436378479]
[-0.942713737487793]
[-0.964977502822876]
[-0.6798200607299805]
[-0.7721402645111084]
[-0.9784448146820068]
[-0.6930570602416992]
[-0.6638383865356445]
[-0.7550640106201172]
[-1.0940537452697754]
[-0.7436025142669678]
[-1.0623281002044678]
[-0.46306920051574707]
[-0.6948075294494629]
[-0.33553266525268555]
[-1.2106492519378662]
[-0.7297763824462891]
[-0.8717366456985474]
[-1.0033198595046997]
[-1.204565167427063]
[-1.1965603828430176]
[-0.7938063144683838]
[-1.0171314477920532]
[-0.8316192626953125]
[-0.6358418464660645]
[-0.9341994524002075]
[-1.17668819

In [12]:
def save_models():

        with session.as_default():
                with session.graph.as_default():        

                    timestamp = int(datetime.datetime.now().timestamp())

                    actor.model.save_weights(model_storage_path + "/actormodel.h5", overwrite=True)
                    with open(model_storage_path + "/actormodel.json", "w") as outfile:
                        json.dump(actor.model.to_json(), outfile)

                    critic.model.save_weights(model_storage_path + "/criticmodel.h5", overwrite=True)
                    with open(model_storage_path + "/criticmodel.json", "w") as outfile:
                        json.dump(critic.model.to_json(), outfile)

                    actor.target_model.save_weights(model_storage_path + "/actortargetmodel.h5", overwrite=True)
                    with open(model_storage_path + "/actormodeltarget.json", "w") as outfile:
                        json.dump(actor.target_model.to_json(), outfile)

                    critic.target_model.save_weights(model_storage_path + "/critictargetmodel.h5", overwrite=True)
                    with open(model_storage_path + "/criticmodeltarget.json", "w") as outfile:
                        json.dump(critic.target_model.to_json(), outfile)

                    print("Models Saved Successfully !!!")


In [13]:
save_models()

Models Saved Successfully !!!


/home/sanjay/anaconda3/envs/powertac_ddpg/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
